In [1]:
import battery_parser as bp
import os
import pandas as pd
import numpy as np
import re
import copy
import itertools

In [2]:

root_dir  = r'D:\!Science\Физтех\Циклирования\28.05.24 - результаты'



In [3]:
files = bp.list_files(directory=root_dir, filetype='xlsx') #TODO добавить ключ, по которому можно будет регулировать глубину погружения


In [4]:
patterns = [r'([A-Z]{3})', r'([^\\]*)', r'([0-9][^\\]*)', r'.?([0-9]?[0-9])[^\\]*', r'.*-(13[0-9])-.-([0-9]).*-([0-9]{14})']
column_names = ['Chem','Tech', 'Step', 'Pouch','Potentiostat', 'Channel', 'Datetime_id']
pattern = r'\\'.join(patterns)
print(pattern)
parser = bp.importing.Regex_parse()
result = parser(strings=files,pattern=pattern,column_names= column_names)

([A-Z]{3})\\([^\\]*)\\([0-9][^\\]*)\\.?([0-9]?[0-9])[^\\]*\\.*-(13[0-9])-.-([0-9]).*-([0-9]{14})
Warning! For 
 D:\!Science\Физтех\Циклирования\28.05.24 - результаты\LFP\Заявка №73 Старение+45 2-3\4. п.5 Циклирование 28 дней\разряд Приложение Б\BTS82-133-1-7-99-20240416085854-Разряд профилем Б.xlsx 
 found 0 entries!
Warning! For 
 D:\!Science\Физтех\Циклирования\28.05.24 - результаты\LFP\Заявка №73 Старение+45 2-3\4. п.5 Циклирование 28 дней\разряд Приложение Б\BTS82-133-1-8-99-20240416085912-Разряд профилем Б.xlsx 
 found 0 entries!


In [5]:
type_convert = {i:str for i in result.columns}
result = result.astype({'Pouch':int}, copy=True)
result = result.astype(type_convert, copy=True)

result['ID'] = result['Chem']+result['Pouch']
result.insert(0, 'ID', result.pop('ID'))


thermos ={'1':['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
          '2':['LFP2', 'LFP3', 'NMC6', 'NMC7', 'LTO11', 'LTO12'],
          '3':['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7']}
protocol = {'gost45':['NMC6', 'NMC7', 'LTO11', 'LTO12', 'LFP2', 'LFP3'],
          'gost60':['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7'],
          'Imax45':['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
          'CCroom':['NMC10', 'NMC11', 'LTO4', 'LTO5', 'LFP1', 'LFP8']}


In [6]:
if bp.modifications.check_dict_intersection(thermos) or bp.modifications.check_dict_intersection(protocol):
    print('Intersection!')

class Find_key:
    def __init__(self, check_dict):
        self.dictionary = check_dict


    def __call__(self, element):
        result = [key for key, value in self.dictionary.items() if element in value]
        if len(result) == 1:
            return result[0]
        return result

thermos_creator = Find_key(thermos)
technique_creator = Find_key(protocol)

In [7]:
result['Protocol'] = result['ID'].apply(technique_creator)
result.insert(2, 'Protocol', result.pop('Protocol'))
result['Thermos'] = result['ID'].apply(thermos_creator)
result.insert(6, 'Thermos', result.pop('Thermos'))


In [8]:
result.sort_values('Step', inplace=True)

In [9]:
step_id = [bp.modifications.step_id_creator(pouch) for name, pouch in result.groupby('ID')]
result['Step_id'] = pd.concat(step_id, )
result.insert(4, 'Step_id', result.pop('Step_id'))

In [10]:
result_path = os.path.join(r'D:\!Science\Физтех\Циклирования\Результаты', 'File_statistics.csv')
bp.exporting.save_experiment(result, result_path)
